# Description

Notebook 1/2 du groupe 6 (Achille, Benjamin, Chedha, Hubert, Victor)

1_Panier_fearures est un notebook qui preprocess la base de données transactionnelle.

### Dans un premier temps:

- toutes les transactions vont être regroupées par paniers. L'idée est de pouvoir rassembler les achats qui se sont faits le même jour et au même endroit par un client.

- On va également calculer la différence de temps entre la date de facturation et la date de commande, une grande différence en nombre de jours peut être la conséquence de retards de livraison.

### Dans un second temps:

- on détaille plus les informations sur les produits en matière de quantité totale achetés par panier et de nombre de commandes effectués.

### Dans un dernier temps:

- on compte le nombre de channels différents qu'un client utilise pour commander

Le résultat final est stocké dans "panier client full.csv' afin de pouvoir le traiter dans notre deuxième notebook pour l'analyse des churns.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

chunksize = 5e+6
dtypes = {"order_channel": "category", 'product_id' : np.int32, "client_id" : np.int32, "sales_net" : np.float32,\
     "quantity" : np.int32, 'branch_id' : np.int16}
filename = 'dataset_sales_2.csv'

def load_df(filename):
    data = pd.read_csv(filename, sep = ';', parse_dates = ['date_order', 'date_invoice'],\
                                                                        chunksize = chunksize, dtype = dtypes)
    return data

In [2]:
#Fonction de concatenation des dataframe calculés par channel
#Note: les chunksize ont été prix assez gros afin de limiter les effets de bords
def concat_all(filename, channel, parse_dates = ['date_order']):
    full = []
    for ch in channel:
        df = pd.read_csv(filename + '_' + ch + '.csv', parse_dates = parse_dates)
        df['channel'] = ch
        full.append(df)
        
    full_df = pd.concat(full)
    full_df.to_csv(filename + '_full.csv', index = False)
    
    return full_df

# Par panier

## Calcul des paniers moyens et du nombre de produits "delayed"

In [3]:
def delay_calc(df):
    df['delay'] = (df.date_invoice - df.date_order).map(lambda x: x.days)
    df['delay_bool'] = df.delay > 0
    df['delay'] = df['delay'].replace({0: np.NaN})
    return df

In [4]:
subset_delay = ['client_id', 'date_order', 'sales_net', 'quantity', 'delay', 'delay_bool']
agg_dict_delay = {'sales_net': 'sum', 'quantity' : 'sum', 'number_of_products' : 'sum',
                  'delay' : ['mean', 'max'], 'delay_bool' : 'sum'}
agg_dict_results = {'sales_net': 'sum', 'quantity' : 'sum', 'number_of_products' : 'sum',
                  'delay_mean' : 'mean', 'delay_max' : 'max', 'delay_bool' : 'sum'}
group_list_delay = ['client_id', 'date_order']
order_channel = ['at the store', 'online', 'by phone', 'other', 'during the visit of a sales rep']

def panier_moyen(df, subset, agg_dict, group_list, channel):
    #Calcul des delay pour toutes les transactions
    df = df[df['order_channel'] == channel].copy(deep = True)
    df = delay_calc(df).loc[:, subset]
    
    #Ajout de la dimension pour calculer le nombre de produits
    df['number_of_products'] = 1
    df = df.groupby(group_list).agg(agg_dict)
    
    #Renomage des colonnes afin de ne pas avoir le multi-level index
    df.columns = list(map(lambda x: '_'.join(x) if (x[0] == 'delay') else x[0], df.columns.values))
    return df

In [5]:
def panier_client(filename, subset, agg_dict, agg_dict_results, group_list):
    for order_ch in order_channel:
        i = 0
        print(order_ch)
        data = load_df(filename)
        for chunk in data:
            t = time.time()
            df = panier_moyen(chunk, subset, agg_dict, group_list, order_ch)
            df.reset_index(inplace = True)
            print('[%d] 1st step : %.4f' %(i, (time.time() - t)))
            
            if i == 0:
                results = pd.DataFrame(columns = df.columns, dtype = float)

            t = time.time() 
            results = pd.concat([results, df]).groupby(group_list).agg(agg_dict_results).reset_index()
            print('[%d] 2nd step : %.4f' %(i, (time.time() - t)))

            i = i+1
        results = results.fillna(0)
        results.to_csv('panier_client_' + order_ch + '.csv', index = False)
    return results

In [6]:
results_panier = panier_client(filename, subset_delay, agg_dict_delay, agg_dict_results, group_list_delay)

at the store
[0] 1st step : 23.2041
[0] 2nd step : 0.2428
[1] 1st step : 22.1235
[1] 2nd step : 0.4796
[2] 1st step : 22.1130
[2] 2nd step : 0.6239
[3] 1st step : 22.0012
[3] 2nd step : 0.8385
[4] 1st step : 22.9221
[4] 2nd step : 1.1402
[5] 1st step : 1197.7708
[5] 2nd step : 1.7353
[6] 1st step : 23.3496
[6] 2nd step : 1.7587
[7] 1st step : 22.7701
[7] 2nd step : 1.8146
[8] 1st step : 23.4631
[8] 2nd step : 2.4405
[9] 1st step : 23.6036
[9] 2nd step : 2.8656
[10] 1st step : 22.3804
[10] 2nd step : 2.9131
[11] 1st step : 21.7534
[11] 2nd step : 3.1966
[12] 1st step : 14.5806
[12] 2nd step : 3.2395
online
[0] 1st step : 3.6403
[0] 2nd step : 0.0313
[1] 1st step : 3.6582
[1] 2nd step : 0.0529
[2] 1st step : 4.1630
[2] 2nd step : 0.0878
[3] 1st step : 4.1856
[3] 2nd step : 0.1230
[4] 1st step : 3.8003
[4] 2nd step : 0.1372
[5] 1st step : 3.6947
[5] 2nd step : 0.1438
[6] 1st step : 3.7501
[6] 2nd step : 0.1671
[7] 1st step : 4.0105
[7] 2nd step : 0.2240
[8] 1st step : 4.1047
[8] 2nd step 

In [7]:
full = concat_all('panier_client', order_channel)

In [8]:
full.head()

,client_id,date_order,sales_net,quantity,number_of_products,delay_mean,delay_max,delay_bool,channel
0,6.0,2017-11-29,204.12961,8.0,2.0,0.0,0.0,0.0,at the store
1,6.0,2018-11-20,270.80200,20.0,4.0,0.0,0.0,0.0,at the store
2,6.0,2018-11-22,193.42080,3.0,1.0,0.0,0.0,0.0,at the store
3,6.0,2019-02-12,139.24200,3.0,1.0,0.0,0.0,0.0,at the store
4,6.0,2019-02-13,35.13480,6.0,2.0,0.0,0.0,0.0,at the store


# Est ce que les branch ID sont tous les mêmes sur un panier client ? 

In [9]:
# Tests, non utile pour générer le csv final

if False:
    data = load_df(filename)
    df = data.get_chunk()
    
    analyse = df.groupby(['client_id', 'date_order', 'order_channel']).agg({'branch_id' : ['count', 'min']})
    
    analyse.dropna()
    
    analyse.shape 
    # Non ce n'est pas le cas, plusieurs branch ID pour un même panier client
    # (panier définit par un client_id, date_order, order_channel)

# Visions produits

In [10]:
subset_products = ['client_id', 'product_id', 'quantity']
agg_dict_products = {'quantity' : 'sum'}
group_list_products = ['client_id', 'product_id']

def product_client(filename, subset, agg_dict, group_list, order_channel):
    for order_ch in order_channel:
        i = 0        
        print(order_ch)
        data = load_df(filename)
        for chunk in data:
            t = time.time()    
            df = chunk[chunk['order_channel'] == order_ch].copy(deep = True)
            df = df.loc[:, subset]
            df = df.groupby(group_list).agg(agg_dict)
            df.reset_index(inplace = True)
            print('[%d] 1st step : %.4f' %(i, (time.time() - t)))

            if i == 0:
                results = pd.DataFrame(columns = df.columns, dtype = float)

            t = time.time() 
            results = pd.concat([results, df]).groupby(group_list).agg(agg_dict).reset_index()
            print('[%d] 2nd step : %.4f' %(i, (time.time() - t)))

            i = i+1

        results = results.groupby(group_list).agg({'quantity': 'count'}).reset_index()
        results.to_csv('product_client_' + order_ch + '.csv', index = False)
    return results

In [11]:
results_products = product_client(filename, subset_products, agg_dict_products, group_list_products, order_channel)

at the store
[0] 1st step : 1.1786
[0] 2nd step : 0.7852
[1] 1st step : 1.0811
[1] 2nd step : 1.4357
[2] 1st step : 1.0460
[2] 2nd step : 1.9096
[3] 1st step : 1.0568
[3] 2nd step : 2.2951
[4] 1st step : 1.0379
[4] 2nd step : 3.3862
[5] 1st step : 1.0377
[5] 2nd step : 3.6953
[6] 1st step : 1.0468
[6] 2nd step : 4.5499
[7] 1st step : 1.0302
[7] 2nd step : 4.5109
[8] 1st step : 1.0214
[8] 2nd step : 4.9837
[9] 1st step : 0.9890
[9] 2nd step : 5.6309
[10] 1st step : 1.3242
[10] 2nd step : 5.9273
[11] 1st step : 0.9885
[11] 2nd step : 7.0281
[12] 1st step : 0.5643
[12] 2nd step : 7.4115
online
[0] 1st step : 0.1895
[0] 2nd step : 0.1509
[1] 1st step : 0.1878
[1] 2nd step : 0.1739
[2] 1st step : 0.3820
[2] 2nd step : 0.4315
[3] 1st step : 0.2674
[3] 2nd step : 0.3003
[4] 1st step : 0.2648
[4] 2nd step : 0.4695
[5] 1st step : 0.1937
[5] 2nd step : 0.4421
[6] 1st step : 0.2101
[6] 2nd step : 0.9016
[7] 1st step : 0.2107
[7] 2nd step : 0.6528
[8] 1st step : 0.2068
[8] 2nd step : 0.7134
[9] 1s

In [12]:
full_products = concat_all('product_client', order_channel, parse_dates = False)

In [13]:
full_products.head()

,client_id,product_id,quantity,channel
0,6.0,46698.0,1,at the store
1,6.0,50772.0,1,at the store
2,6.0,113967.0,1,at the store
3,6.0,169851.0,1,at the store
4,6.0,654297.0,1,at the store


# Nombre de branch unique par client 

In [14]:
subset_branch = ['client_id', 'branch_id', 'quantity']
agg_dict_branch = {'quantity' : 'sum'}
group_list_branch = ['client_id', 'branch_id']

def branch_client(filename, subset, agg_dict, group_list):
    i = 0        
    data = load_df(filename)
    for chunk in data:
        t = time.time()    
        df = chunk.copy(deep = True)
        df = df.loc[:, subset]
        df = df.groupby(group_list).agg(agg_dict)
        df.reset_index(inplace = True)
        print('[%d] 1st step : %.4f' %(i, (time.time() - t)))

        if i == 0:
            results = pd.DataFrame(columns = df.columns, dtype = float)

        t = time.time() 
        results = pd.concat([results, df]).groupby(group_list).agg(agg_dict).reset_index()
        print('[%d] 2nd step : %.4f' %(i, (time.time() - t)))

        i = i+1

    results = results.groupby(group_list).agg({'quantity': 'count'}).reset_index()
    results.to_csv('branch_client.csv', index = False)
    return results

In [15]:
results_branch = branch_client(filename, subset_branch, agg_dict_branch, group_list_branch)

[0] 1st step : 0.5064
[0] 2nd step : 0.0698
[1] 1st step : 0.4904
[1] 2nd step : 0.0845
[2] 1st step : 0.4631
[2] 2nd step : 0.0982
[3] 1st step : 0.4574
[3] 2nd step : 0.1063
[4] 1st step : 0.4920
[4] 2nd step : 0.1131
[5] 1st step : 0.4687
[5] 2nd step : 0.1189
[6] 1st step : 0.4751
[6] 2nd step : 0.1254
[7] 1st step : 0.4589
[7] 2nd step : 0.1262
[8] 1st step : 0.5115
[8] 2nd step : 0.1409
[9] 1st step : 0.4995
[9] 2nd step : 0.1589
[10] 1st step : 0.5372
[10] 2nd step : 0.1647
[11] 1st step : 0.6499
[11] 2nd step : 0.1548
[12] 1st step : 0.3452
[12] 2nd step : 0.1616
